In [ ]:
import os
import csv

INPUT_CSV  = r"input_refs.csv"   # CSV con columna PartNumber
OUTPUT_DIR = r"\partes_txt"       # Carpeta para part1.txt, part2.txt...
CHUNK_SIZE = 200

os.makedirs(OUTPUT_DIR, exist_ok=True)

# Leer referencias
parts = []
with open(INPUT_CSV, "r", encoding="utf-8-sig") as f:
    reader = csv.DictReader(f)
    if "PartNumber" not in reader.fieldnames:
        raise ValueError("El CSV debe tener columna 'PartNumber'")
    for row in reader:
        if row["PartNumber"]:
            parts.append(row["PartNumber"].strip())

# Partir en chunks de 200
for i in range(0, len(parts), CHUNK_SIZE):
    chunk = parts[i:i+CHUNK_SIZE]
    out_file = os.path.join(OUTPUT_DIR, f"part{i//CHUNK_SIZE + 1}.txt")
    with open(out_file, "w", encoding="utf-8") as f:
        f.write("\n".join(chunk))
    print(f"[OK] Generado: {out_file} con {len(chunk)} referencias")

[OK] Generado: \partes_txt\part1.txt con 200 referencias
[OK] Generado: \partes_txt\part2.txt con 200 referencias
[OK] Generado: \partes_txt\part3.txt con 200 referencias
[OK] Generado: \partes_txt\part4.txt con 200 referencias
[OK] Generado: \partes_txt\part5.txt con 200 referencias
[OK] Generado: \partes_txt\part6.txt con 200 referencias
[OK] Generado: \partes_txt\part7.txt con 200 referencias
[OK] Generado: \partes_txt\part8.txt con 200 referencias
[OK] Generado: \partes_txt\part9.txt con 200 referencias
[OK] Generado: \partes_txt\part10.txt con 200 referencias
[OK] Generado: \partes_txt\part11.txt con 200 referencias
[OK] Generado: \partes_txt\part12.txt con 200 referencias
[OK] Generado: \partes_txt\part13.txt con 200 referencias
[OK] Generado: \partes_txt\part14.txt con 200 referencias
[OK] Generado: \partes_txt\part15.txt con 200 referencias
[OK] Generado: \partes_txt\part16.txt con 200 referencias
[OK] Generado: \partes_txt\part17.txt con 200 referencias
[OK] Generado: \partes_

In [1]:
# ============================================================
# 01 - Importación de librerías
# ============================================================
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import os, time, sys, json


# ============================================================
# 02 - Cargar configuración desde JSON
# ============================================================
CONFIG_PATH = "ConfigWBUSA.json"

if not os.path.isfile(CONFIG_PATH):
    raise FileNotFoundError(f"[ERROR] No se encontró el archivo {CONFIG_PATH}")

with open(CONFIG_PATH, "r", encoding="utf-8") as f:
    config = json.load(f)

USER = config["USER"]
PWD  = config["PWD"]
LOGIN_URL = config["LOGIN_URL"]
CSPS_URL  = config["CSPS_URL"]


# ============================================================
# 03 - Configuración de carpetas de entrada/salida
# ============================================================
# Carpeta con los archivos .txt que contienen Part Numbers
PARTS_DIR  = os.path.abspath(r"C:\partes_txt")

# Carpeta donde se guardarán las descargas
OUTPUT_DIR = r"C:\descargas"


# ============================================================
# 04 - Configuración del navegador
# ============================================================
options = Options()
options.add_argument("--start-maximized")
prefs = {"download.default_directory": OUTPUT_DIR}
options.add_experimental_option("prefs", prefs)

driver = webdriver.Chrome(options=options)
wait = WebDriverWait(driver, 30)


# ============================================================
# 05 - Funciones utilitarias
# ============================================================
def safe_find(by, selector, timeout=30):
    """Busca un elemento en la página asegurando que esté presente."""
    return WebDriverWait(driver, timeout).until(
        EC.presence_of_element_located((by, selector))
    )

def safe_click(by, selector, timeout=30):
    """Hace clic en un elemento asegurando que esté listo para interactuar."""
    el = WebDriverWait(driver, timeout).until(
        EC.element_to_be_clickable((by, selector))
    )
    driver.execute_script("arguments[0].scrollIntoView({block:'center'});", el)
    el.click()
    return el

def switch_to_frame_by_name(name, timeout=30):
    """Cambia al frame indicado (ej. 'menu', 'detail')."""
    WebDriverWait(driver, timeout).until(
        EC.frame_to_be_available_and_switch_to_it((By.NAME, name))
    )

def back_to_default():
    """Regresa al contexto principal (fuera de los frames)."""
    driver.switch_to.default_content()


# ============================================================
# 06 - Navegación en el portal
# ============================================================
def open_multi_part_stock_inquiry():
    """Navega en el menú hasta 'Multi Part Stock Inquiry'."""
    wait.until(EC.presence_of_element_located((By.TAG_NAME, "frameset")))
    switch_to_frame_by_name("menu", 20)
    actions = ActionChains(driver)

    # Expandir Part Master Data (puede variar el id, en muchos casos es folderIcon14)
    try:
        expand_icon = safe_click(By.ID, "folderIcon14", 10)
        actions.move_to_element(expand_icon).pause(0.2).perform()
    except Exception:
        pass  # ya estaba abierto o el id cambió

    # Clic en el link "Multi Part Stock Inquiry"
    link = WebDriverWait(driver, 20).until(
        EC.element_to_be_clickable((By.XPATH, "//a[@title='Multi Part Stock Inquiry']"))
    )
    actions.move_to_element(link).pause(0.2).click().perform()

    # Pasar al frame de detalle
    back_to_default()
    switch_to_frame_by_name("detail", 20)
    safe_find(By.ID, "filePath", 20)

def do_login_and_open_multi_inquiry():
    """Hace login en el portal y abre el módulo Multi Part Stock Inquiry."""
    driver.get(LOGIN_URL)
    safe_find(By.ID, "userID", 20).send_keys(USER)
    driver.find_element(By.ID, "password").send_keys(PWD)
    driver.find_element(By.ID, "btn-submit").click()

    # Esperar a que redireccione al portal
    wait.until(EC.url_contains("my.dlrportal.com"))

    # Abrir CSPS en nueva pestaña
    driver.switch_to.new_window('tab')
    driver.get(CSPS_URL)

    # Entrar al módulo
    open_multi_part_stock_inquiry()


# ============================================================
# 07 - Flujo principal
# ============================================================
try:
    # Validar carpetas
    if not os.path.isdir(PARTS_DIR):
        print(f"[ERROR] No se encuentra la carpeta con los .txt: {PARTS_DIR}")
        sys.exit(1)

    os.makedirs(OUTPUT_DIR, exist_ok=True)

    # Login y acceso al módulo
    do_login_and_open_multi_inquiry()

    # Iterar por cada archivo .txt
    for fname in sorted(os.listdir(PARTS_DIR)):
        if not fname.endswith(".txt"):
            continue

        file_path = os.path.join(PARTS_DIR, fname)
        print(f"[INFO] Subiendo {file_path}")

        # Subir archivo
        file_input = safe_find(By.ID, "filePath", 20)
        file_input.send_keys(file_path)

        # Click en Search
        safe_click(By.NAME, "Search", 20)

        # Esperar resultados
        wait.until(EC.presence_of_element_located(
            (By.XPATH, "//td[contains(., 'Part Number')]")
        ))
        time.sleep(1.5)

        # Descargar Excel
        safe_click(By.NAME, "Download", 20)
        print(f"[OK] Descargando resultado de {fname} en {OUTPUT_DIR}")

        time.sleep(6)  # darle tiempo a la descarga

    print("[OK] Todas las partes procesadas.")

except Exception as e:
    print("[ERROR]", repr(e))

finally:
    try:
        input("Pulsa ENTER para cerrar...")
    except Exception:
        pass
    driver.quit()

[ERROR] NoSuchWindowException()


In [ ]:
import os
import xlrd
from openpyxl import Workbook

# Ruta de la carpeta de descargas
carpeta = r"C:\descargas"
archivos = [f for f in os.listdir(carpeta) if f.endswith(".xls")]

# Crear un archivo Excel nuevo
wb = Workbook()
ws = wb.active
ws.title = "Combinado"

fila_actual = 1  # para llevar control de en qué fila vamos

for archivo in archivos:
    ruta = os.path.join(carpeta, archivo)
    print(f"Procesando: {archivo}")

    # Abrir .xls con xlrd
    libro = xlrd.open_workbook(ruta)
    hoja = libro.sheet_by_index(0)  # primera hoja

    # Recorrer filas y celdas
    for r in range(hoja.nrows):
        valores = []
        for c in range(hoja.ncols):
            valores.append(hoja.cell_value(r, c))

        # Escribir en el Excel final
        for col, valor in enumerate(valores, start=1):
            ws.cell(row=fila_actual, column=col, value=valor)

        fila_actual += 1  # siguiente fila

# Guardar archivo combinado
salida = os.path.join(carpeta, "Combinado.xlsx")
wb.save(salida)
print(f"✅ Todos los .xls combinados en: {salida}")